In [1]:
import pandas as pd
import statistics
import sqlite3
import locale
from decimal import Decimal

In [2]:
# Criar uma conexão com o banco de dados SQLite (ou criar um novo arquivo se ele não existir)
conn = sqlite3.connect("petrobras.db")

# Criar um cursor para executar comandos SQL
cursor = conn.cursor()

# Criar uma tabela no banco de dados (substitua isso com sua própria estrutura)
cursor.execute("""
    CREATE TABLE pj (
        Cnpj INTEGER PRIMARY KEY NOT NULL,
        Nome TEXT NOT NULL,
        Cidade TEXT NOT NULL,
        Regiao TEXT NOT NULL,
        Uf TEXT NOT NULL,
        Cep INTEGER NOT NULL
    )
""")

cursor.execute("""
    CREATE TABLE tabela_produtos (
        ID INTEGER PRIMARY KEY NOT NULL,
        Empresa_CNPJ INTEGER,
        Produto TEXT NOT NULL,
        Data_Coleta DATE NOT NULL,
        Valor_Venda REAL NOT NULL,
        Valor_Compra REAL NOT NULL,
        Unidade_Medida TEXT NOT NULL,
        Bandeira TEXT NOT NULL,
        FOREIGN KEY (Empresa_CNPJ) REFERENCES pj(cnpj)
);

""")

OperationalError: table pj already exists

In [4]:
pathExcel = "C:\\Users\\IMendes_41\\OneDrive\\Preto-Bras\\BaseDados\\Relatório_Combustiveis.xlsx"
        
dt = pd.read_excel(pathExcel,  dtype={'cnpj': str})

dt.to_sql("pj", conn, if_exists="replace", index=False)

14908

In [26]:
pathExcel = ".\\BaseDados\Vendas.xlsx"
        
dt = pd.read_excel(pathExcel,  dtype={'cnpj': str})

dt.to_sql("tabela_produtos", conn, if_exists="replace", index=False)

101006

In [27]:
consulta = """SELECT *
FROM tabela_produtos
"""
# Exibir os resultados da consulta
resultados = pd.read_sql(consulta, conn)
resultados

,id,cnpj,produto,data,venda,compra,tipo,bandeira
0,1,00003188000121,GASOLINA,1167879600,2.360,2.0560,R$ / litro,BRANCA
1,2,00003188000121,ETANOL,1167879600,1.160,0.9887,R$ / litro,BRANCA
2,3,00003188000121,DIESEL,1167879600,1.790,1.6250,R$ / litro,BRANCA
3,4,34274233001338,GASOLINA,1167879600,2.499,2.2348,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
4,5,34274233001338,ETANOL,1167879600,1.699,1.4842,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
...,...,...,...,...,...,...,...,...
101001,101002,81844789000169,GASOLINA,1169434800,2.650,2.1000,R$ / litro,BRANCA
101002,101003,81844789000169,ETANOL,1169434800,1.799,1.5400,R$ / litro,BRANCA
101003,101004,81844789000169,DIESEL,1169434800,1.960,1.7249,R$ / litro,BRANCA
101004,101005,80114879000212,GASOLINA,1169434800,2.667,2.1885,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.


In [12]:
def FormatNumber(number):
        from locale import setlocale, format_string, LC_ALL
        
        setlocale(LC_ALL, '')
        number = format_string('%d', int(number), grouping=True)

        num_pontos = number.count('.')
        splitNumber = str(number).split('.')

        match num_pontos:
            case 1:
                number = splitNumber[0] + ' Mil'
                return number
            case 2:
                number = splitNumber[0] + ' Mi'
                return number
            case _:
                return number

In [13]:
# -------------------------------------------------------------------------
#                           FATURAMENTO
# -------------------------------------------------------------------------

consulta = f"SELECT SUM(venda) FROM tabela_produtos"

# Executar a consulta
cursor.execute(consulta)

# Recuperar os resultados
resultados = cursor.fetchall()

# Exibir os nomes das colunas
faturamento = resultados[0][0]

print(FormatNumber(faturamento))

207 Mil


In [14]:
# -------------------------------------------------------------------------
#                           LUCRO
# -------------------------------------------------------------------------

consulta = f"SELECT SUM(venda - compra) FROM tabela_produtos"

# Executar a consulta
cursor.execute(consulta)

# Recuperar os resultados
resultados = cursor.fetchall()

locale.setlocale(locale.LC_ALL, '')

# Exibir os nomes das colunas
lucro = resultados[0][0]

print(FormatNumber(lucro))


28 Mil


In [15]:
# -------------------------------------------------------------------------
#                           QUANTIDADE DE POSTOS
# -------------------------------------------------------------------------
consulta = f"SELECT COUNT(cnpj) FROM pj"

# Executar a consulta
cursor.execute(consulta)

# Recuperar os resultados
resultados = cursor.fetchall()

contagemPostos = resultados[0][0]

print(FormatNumber(contagemPostos))


14 Mil


In [16]:
# -------------------------------------------------------------------------
#                           ESTADO MAIS LUCRATIVO
# -------------------------------------------------------------------------

consulta =  f"""SELECT SUM(t1.venda - t1.compra),
                    t2.uf
                FROM tabela_produtos t1
                INNER JOIN pj t2 ON t1.cnpj = t2.Cnpj
                GROUP BY 2
                ORDER BY 1 DESC
                LIMIT 1"""

# Executar a consulta
cursor.execute(consulta)

# Recuperar os resultados
resultados = cursor.fetchall()

estado = resultados[0][1]

In [68]:
# -------------------------------------------------------------------------
#                           GRAFICO DE LINHAS
# -------------------------------------------------------------------------

consulta =  f"""SELECT SUM(venda - compra),
                    strftime('%d', data, 'unixepoch')
                FROM tabela_produtos
                GROUP BY 2
                ORDER BY 2
                """

# Executar a consulta
cursor.execute(consulta)

# Recuperar os resultados
resultados = cursor.fetchall()

lucro = []
dia = []

for resultado in resultados:
    valor = FormatNumber(resultado[0])
    lucro.append(valor)
    dia.append(int(resultado[1]))

lucro


['2 Mil',
 '4 Mil',
 '1 Mil',
 '1 Mil',
 '2 Mil',
 '4 Mil',
 '45',
 '1 Mil',
 '2 Mil',
 '3 Mil',
 '726',
 '472',
 '958',
 '1 Mil',
 '993']

In [66]:
# -------------------------------------------------------------------------
#                           GRAFICO DE BARRAS
# -------------------------------------------------------------------------

consulta =  f"""SELECT SUM(t1.venda - t1.compra) as lucro,
                    t2.nome
                FROM tabela_produtos t1
                INNER JOIN pj t2 ON t1.cnpj = t2.Cnpj
                GROUP BY 2
                ORDER BY 1 DESC
                LIMIT 10"""

# Executar a consulta
cursor.execute(consulta)

# Recuperar os resultados
resultados = cursor.fetchall()

lucro = []
empresa = []

for resultado in resultados:
    valor = FormatNumber(resultado[0])
    lucro.append(valor)

    empresaSplit = resultado[1].split()
    razaoSocial = ' '.join(empresaSplit[:2])
    empresa.append(razaoSocial)


['COMERCIAL SANTA',
 'JOSE OTONIO',
 'CARREFOUR COMERCIO',
 'SETE ESTRELAS',
 'ARARA AZUL',
 'POSTOS LIBERDADE',
 'ABASTECEDORA ABM',
 'PETROBRAS DISTRIBUIDORA',
 'COMERCIAL BUFFON',
 'SOBRAL &']